In [2]:
import pandas as pd

In [3]:
import torch

In [4]:
torch.cuda.is_available()

True

In [5]:
df = pd.read_csv("./ToN_IoT_training_set.csv")

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
x = df
y = df['type']
x, _x, y, _y = train_test_split(x, y, random_state=189, test_size=0.2, stratify=y)

In [8]:
x.to_csv('./training-set_0.csv', index=False)

In [9]:
_x.to_csv('./training-set_1.csv', index=False)

In [10]:
df['type'].value_counts()

type
normal        34955
backdoor      14102
dos           14094
password      14028
ransomware    13993
ddos          13981
injection     13975
xss           13974
scanning      13907
mitm            721
Name: count, dtype: int64

In [11]:
from ctgan import CTGAN

In [12]:
ctgan = CTGAN(verbose=True, epochs=100)

In [13]:
train_data = _x

In [14]:
ctgan_data = train_data[1:1]
for val in train_data['type'].unique():
    val_data = train_data[train_data['type'] == val].drop_duplicates()
    if len(val_data) > 200:
        val_data = val_data.sample(n=200, random_state=42)
    ctgan_data = pd.concat([ctgan_data, val_data])

In [15]:
ctgan.fit(ctgan_data, ctgan_data.columns)

Gen. (-1.60) | Discrim. (0.36):  35%|███▌      | 35/100 [00:32<01:00,  1.07it/s] 

In [ ]:
data = train_data[1:1]

In [ ]:
data = pd.concat([data, ctgan.sample(n=100, condition_column='type', condition_value='normal')]) 

In [ ]:
data

,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,...,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,192.168.1.33,41878,192.168.1.37,5357,tcp,http,0.128041,80,3653,S1,...,0,0,-,-,-,-,46,-,0,ransomware
1,192.168.1.31,91,151.101.30.49,6667,udp,ssl,0.047484,155,149,RSTR,...,0,0,-,-,-,-,-,-,0,mitm
2,192.168.1.79,56936,151.101.30.49,10502,udp,-,0.000054,3010,216,OTH,...,0,0,-,-,-,DNS_RR_unknown_type,-,-,1,normal
3,fe80::855:e4ef:de26:7aca,34296,216.58.203.109,443,tcp,dce_rpc,125.065890,37,53,REJ,...,0,0,-,-,-,-,46,-,1,xss
4,192.168.1.32,49535,196.216.169.11,4444,icmp,ssl,0.003205,260,1207,SH,...,0,0,-,-,-,-,-,-,1,scanning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,192.168.1.32,47092,52.139.250.253,1249,tcp,dns,0.080878,170,90,S2,...,0,0,-,-,-,-,-,-,1,backdoor
96,192.168.1.32,62946,216.58.199.34,2920,icmp,-,0.042546,235,463,OTH,...,0,0,-,-,-,-,-,-,0,xss
97,192.168.1.31,60834,196.216.169.11,993,tcp,-,60.760660,239,5884,S1,...,0,0,-,-,-,DNS_RR_unknown_type,-,F,0,ransomware
98,192.168.1.33,53988,192.168.1.79,10502,icmp,http,0.086189,2542,1342,S3,...,0,0,-,-,-,-,-,-,1,dos
